In [ ]:
!pip install langchain
!pip install langchain-chroma
!pip install langchain_groq

In [2]:
from langchain_core.documents import Document

document = [
    Document(
        page_content = "Dogs are great companions, known for their loyalty and friendliness.",
        metadata = {"source":"mammal-pets-doc"}
    ),
    Document(
    page_content = "Parrots are intelligent birds that can mimic human speech and form strong bonds with their owners.",
    metadata = {"source":"bird-pets-doc"}
    ),
    Document(
        page_content = "Goldfish are peaceful aquarium pets that can live for many years with proper care.",
        metadata = {"source":"fish-pets-doc"}
    ),
    Document(
        page_content = "Rabbits are social animals that require regular exercise and a diet rich in hay and vegetables.",
        metadata = {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content = "Hamsters are nocturnal rodents that make great first pets for children due to their small size and simple care requirements.",
        metadata = {"source":"mammal-pets-doc"}
    )
]

In [3]:
document

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds that can mimic human speech and form strong bonds with their owners.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are peaceful aquarium pets that can live for many years with proper care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require regular exercise and a diet rich in hay and vegetables.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal rodents that make great first pets for children due to their small size and simple care requirements.')]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key= groq_api_key, model="Llama3-8b-8192")
llm

In [ ]:
!pip install langchain_huggingface

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\Python\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
## VectorStore -> Convert this texts into vectors(embeddings) and then store inside the vector db

from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(document, embedding=embeddings)
vectorstore


In [12]:
vectorstore.similarity_search("cat")

[Document(id='9168eadd-0e2f-404f-878e-2900af47ccf4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b9b8618c-915e-4c20-aeb1-20103d90f4bc', metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal rodents that make great first pets for children due to their small size and simple care requirements.'),
 Document(id='e2598edd-acb1-4de5-a46e-08c2f55cdbf8', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require regular exercise and a diet rich in hay and vegetables.'),
 Document(id='2558382d-aa3d-40bd-a2bf-32b0c6b9ee48', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds that can mimic human speech and form strong bonds with their owners.')]

In [13]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='9168eadd-0e2f-404f-878e-2900af47ccf4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b9b8618c-915e-4c20-aeb1-20103d90f4bc', metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal rodents that make great first pets for children due to their small size and simple care requirements.'),
 Document(id='e2598edd-acb1-4de5-a46e-08c2f55cdbf8', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require regular exercise and a diet rich in hay and vegetables.'),
 Document(id='2558382d-aa3d-40bd-a2bf-32b0c6b9ee48', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds that can mimic human speech and form strong bonds with their owners.')]

In [14]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='9168eadd-0e2f-404f-878e-2900af47ccf4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='b9b8618c-915e-4c20-aeb1-20103d90f4bc', metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal rodents that make great first pets for children due to their small size and simple care requirements.'),
  1.6415950059890747),
 (Document(id='e2598edd-acb1-4de5-a46e-08c2f55cdbf8', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require regular exercise and a diet rich in hay and vegetables.'),
  1.6609950065612793),
 (Document(id='2558382d-aa3d-40bd-a2bf-32b0c6b9ee48', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds that can mimic human speech and form strong bonds with their owners.'),
  1.6684852838516235)]

In [16]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["cat", "dog"])



[[Document(id='9168eadd-0e2f-404f-878e-2900af47ccf4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(id='9168eadd-0e2f-404f-878e-2900af47ccf4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [18]:
## RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("Tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
